PySpark을 로컬머신에 설치하고 노트북을 사용하기 보다는 머신러닝 관련 다양한 라이브러리가 이미 설치되었고 좋은 하드웨어를 제공해주는 Google Colab을 통해 실습을 진행한다.

이를 위해 pyspark과 Py4J 패키지를 설치한다. Py4J 패키지는 파이썬 프로그램이 자바가상머신상의 오브젝트들을 접근할 수 있게 해준다. Local Standalone Spark을 사용한다.

In [3]:
!pip install pyspark==3.3.1 py4j==0.10.9.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 20.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=3b03dd22b40da7b29690a2e6b91f2f6cd401f30e4f5e0cf0a5591bd2813681af
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark


In [8]:
!pip install numpy

     |████████████████████████████████| 14.2 MB 9.9 MB/s eta 0:00:01


In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Boston Housing Linear Regression example") \
    .getOrCreate()

/usr/local/lib/python3.9/dist-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/09/25 14:20:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 보스턴 주택 가격 예측 모델 만들기




In [2]:
spark

In [3]:
!wget https://s3-geospatial.s3-us-west-2.amazonaws.com/boston_housing.csv

--2023-09-25 14:20:54--  https://s3-geospatial.s3-us-west-2.amazonaws.com/boston_housing.csv
Resolving s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)... 52.218.241.209, 52.218.212.25, 52.92.130.2, ...
Connecting to s3-geospatial.s3-us-west-2.amazonaws.com (s3-geospatial.s3-us-west-2.amazonaws.com)|52.218.241.209|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 36240 (35K) [text/csv]
Saving to: ‘boston_housing.csv’

boston_housing.csv  100%[===================>]  35.39K  --.-KB/s    in 0.1s    

2023-09-25 14:20:55 (260 KB/s) - ‘boston_housing.csv’ saved [36240/36240]



In [7]:
!ls -tl

total 40
drwxr-xr-x 1 root root  4096 Jan 26 14:33 sample_data
-rw-r--r-- 1 root root 36240 Jan 31  2021 boston_housing.csv


In [4]:
data = spark.read.csv('./boston_housing.csv', header=True, inferSchema=True)

In [5]:
data.printSchema()

root
 |-- crim: double (nullable = true)
 |-- zn: double (nullable = true)
 |-- indus: double (nullable = true)
 |-- chas: integer (nullable = true)
 |-- nox: double (nullable = true)
 |-- rm: double (nullable = true)
 |-- age: double (nullable = true)
 |-- dis: double (nullable = true)
 |-- rad: integer (nullable = true)
 |-- tax: integer (nullable = true)
 |-- ptratio: double (nullable = true)
 |-- b: double (nullable = true)
 |-- lstat: double (nullable = true)
 |-- medv: double (nullable = true)



In [6]:
data.show()

+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio|     b|lstat|medv|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|
|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5605|  5|311|   15.2| 395.6|12.43|22.9|
|0.14455|12.5| 7.87|   0|0.524|6.172| 96.1|5.9505|  5|311|   15.2| 396.9|19.15|27.1|
|0.21124|12.5| 7.87|   0|0.524|5.631|100.0|6.0821|  5|311|   15.2

## 피쳐 벡터를 만들기

In [9]:
from pyspark.ml.feature import VectorAssembler

feature_columns = data.columns[:-1]
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

In [10]:
feature_columns

['crim',
 'zn',
 'indus',
 'chas',
 'nox',
 'rm',
 'age',
 'dis',
 'rad',
 'tax',
 'ptratio',
 'b',
 'lstat']

In [11]:
data_2 = assembler.transform(data)

In [12]:
data_2.show()

23/09/25 14:21:55 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+--------------------+
|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio|     b|lstat|medv|            features|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+--------------------+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|[0.00632,18.0,2.3...|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|[0.02731,0.0,7.07...|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|[0.02729,0.0,7.07...|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|[0.03237,0.0,2.18...|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33

## 훈련용과 테스트용 데이터를 나누고 Linear Regression 모델을 하나 만든다

In [13]:
train, test = data_2.randomSplit([0.7, 0.3])

In [14]:
from pyspark.ml.regression import LinearRegression

algo = LinearRegression(featuresCol="features", labelCol="medv")
model = algo.fit(train)

23/09/25 14:22:01 WARN Instrumentation: [1488481b] regParam is zero, which might cause numerical instability and overfitting.
23/09/25 14:22:01 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/09/25 14:22:01 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
23/09/25 14:22:01 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


## 모델 성능 측정

In [15]:
evaluation_summary = model.evaluate(test)

In [16]:
evaluation_summary

In [17]:
evaluation_summary.meanAbsoluteError

3.7372681708842

In [18]:
evaluation_summary.rootMeanSquaredError

5.38161657744138

In [19]:
evaluation_summary.r2

0.7165647019604885

## 모델 예측값 살펴보기 

In [20]:
predictions = model.transform(test)

In [27]:
predictions.show()

+-------+----+-----+----+------+-----+----+-------+---+---+-------+------+-----+----+--------------------+------------------+
|   crim|  zn|indus|chas|   nox|   rm| age|    dis|rad|tax|ptratio|     b|lstat|medv|            features|        prediction|
+-------+----+-----+----+------+-----+----+-------+---+---+-------+------+-----+----+--------------------+------------------+
| 0.0136|75.0|  4.0|   0|  0.41|5.888|47.6| 7.3197|  3|469|   21.1| 396.9| 14.8|18.9|[0.0136,75.0,4.0,...|15.218004102485514|
|0.01501|80.0| 2.01|   0| 0.435|6.635|29.7|  8.344|  4|280|   17.0|390.94| 5.99|24.5|[0.01501,80.0,2.0...|27.073058573471883|
|0.01538|90.0| 3.75|   0| 0.394|7.454|34.2| 6.3361|  3|244|   15.9|386.34| 3.11|44.0|[0.01538,90.0,3.7...|  36.5998470052473|
|0.01709|90.0| 2.02|   0|  0.41|6.728|36.1|12.1265|  5|187|   17.0|384.46|  4.5|30.1|[0.01709,90.0,2.0...|25.663278393055624|
|0.02177|82.5| 2.03|   0| 0.415| 7.61|15.7|   6.27|  2|348|   14.7|395.38| 3.11|42.3|[0.02177,82.5,2.0...| 36.31150486

In [25]:
predictions.select(predictions.columns[13:]).show()

+----+--------------------+------------------+
|medv|            features|        prediction|
+----+--------------------+------------------+
|18.9|[0.0136,75.0,4.0,...|15.218004102485514|
|24.5|[0.01501,80.0,2.0...|27.073058573471883|
|44.0|[0.01538,90.0,3.7...|  36.5998470052473|
|30.1|[0.01709,90.0,2.0...|25.663278393055624|
|42.3|[0.02177,82.5,2.0...| 36.31150486208892|
|31.1|[0.02187,60.0,2.9...|31.166712467921126|
|23.9|[0.02543,55.0,3.7...|27.334659727453502|
|34.7|[0.02729,0.0,7.07...|   30.729635231377|
|28.7|[0.02985,0.0,2.18...|25.477085770555853|
|18.5|[0.03041,0.0,5.19...| 19.28144230712698|
|31.2|[0.03049,55.0,3.7...|28.122248151266792|
|34.9|[0.03359,75.0,2.9...|33.031330580480414|
|48.5|[0.0351,95.0,2.68...| 40.88309040659626|
|22.9|[0.03551,25.0,4.8...| 24.60223737223585|
|23.2|[0.03871,52.5,5.3...| 26.40106417357591|
|28.0|[0.04113,25.0,4.8...|28.047444825505575|
|24.8|[0.04297,52.5,5.3...| 26.46091641763198|
|23.9|[0.04462,25.0,4.8...| 26.96987044183468|
|11.9|[0.0474

In [25]:
model.save("boston_housing_model")

In [26]:
!ls boston_housing_model

data  metadata


In [27]:
!ls -tl boston_housing_model

total 8
drwxr-xr-x 2 root root 4096 Jan 30 11:28 data
drwxr-xr-x 2 root root 4096 Jan 30 11:28 metadata


In [28]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [30]:
model_save_name = "boston_housing_model"
path = F"/content/gdrive/My Drive/boston_housing_model" 
model.save(path)

In [31]:
from pyspark.ml.regression import LinearRegressionModel

loaded_model = LinearRegressionModel.load(path)  # "boston_housing_model")

In [32]:
predictions2 = loaded_model.transform(test)

In [33]:
predictions2.select(predictions.columns[13:]).show()

+----+--------------------+------------------+
|medv|            features|        prediction|
+----+--------------------+------------------+
|29.1|[0.01439,60.0,2.9...|31.863971610226315|
|24.5|[0.01501,80.0,2.0...|27.800912626874748|
|44.0|[0.01538,90.0,3.7...|38.081559279682246|
|23.9|[0.02543,55.0,3.7...|27.969747095640912|
|30.8|[0.02763,75.0,2.9...|31.575873666286917|
|25.0|[0.02875,28.0,15....|29.689655131070666|
|28.5|[0.03502,80.0,4.9...|33.855798290686124|
|45.4|[0.03578,20.0,3.3...|39.495929078055646|
|22.0|[0.03932,0.0,3.41...|27.678841182848963|
|33.3|[0.04011,80.0,1.5...|36.633177721713174|
|22.9|[0.04203,28.0,15....| 29.65081329317448|
|18.2|[0.04301,80.0,1.9...|14.312466963142263|
|20.5|[0.04337,21.0,5.6...| 24.52586127306892|
|19.8|[0.04544,0.0,3.24...|21.560592700808435|
|23.3|[0.0456,0.0,13.89...|27.974279873488676|
|17.1|[0.05023,35.0,6.0...|20.027398326196785|
|22.5|[0.05188,0.0,4.49...| 22.19486700208488|
|36.1|[0.05515,33.0,2.1...| 33.36611368642783|
|29.0|[0.0556